In [1]:
import praw

In [2]:
reddit = praw.Reddit(client_id='redacted',
                    client_secret= 'redacted',
                    user_agent='redacted',
                    username='redacted',
                    password='redacted',)

In [3]:
end_of_season_2223 = 1685418710
beginning_of_season2223 = 1659744000


In [4]:
submission_list_liverpool = [
    submission for submission in reddit.subreddit('LiverpoolFC').search('FT') if end_of_season_2223 >= submission.created_utc >= beginning_of_season2223 and submission.link_flair_text == 'Post Match'
]
submission_list_arsenal = [
    submission for submission in reddit.subreddit('Gunners').search('Post Match') if end_of_season_2223 >= submission.created_utc >= beginning_of_season2223 and submission.link_flair_text == 'Post-Match Thread'

]

submission_list_manunited = [
    submission for submission in reddit.subreddit('RedDevils').search('Post Match') if end_of_season_2223 >= submission.created_utc >= beginning_of_season2223

]

submission_list_mancity = [
    submission for submission in reddit.subreddit('MCFC').search('Post') if end_of_season_2223 >= submission.created_utc >= beginning_of_season2223 and submission.link_flair_text == 'Match Day'

]

submission_list_spurs = [
    submission for submission in reddit.subreddit('coys').search('post') if end_of_season_2223 >= submission.created_utc >= beginning_of_season2223 and submission.link_flair_text == 'PostMatch Thread'

]

submission_list_chelsea = [
    submission for submission in reddit.subreddit('chelseafc').search('post') if end_of_season_2223 >= submission.created_utc >= beginning_of_season2223 and submission.link_flair_text == 'Post-Match Thread'

]

In [5]:
import pandas as pd

def get_comment_data(end, beginning, submission_list):
    data = pd.DataFrame()

    for i in submission_list:
        submission = reddit.submission(i)
        finaltitle = i.title

        comment_data = []

        submission.comments.replace_more(limit=None)

        for comment in submission.comments.list():
            comment_data.append(str(comment.body))


        df = pd.DataFrame({finaltitle: comment_data})
        data = pd.concat([data, df], axis=1)
    return data


In [6]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [7]:
def get_mood_ratio(data):

    moodDF = pd.DataFrame()
    #for each column in the data
    for col in data.columns.dropna():
        col_list = []
        #for each comment in each column
        for comment in data[col].dropna():
            #split text into words'
            temp = comment.split(' ')
            #find length
            length = len(temp)
            #need to restrict length of paragraph so that the sentiment pipeline can work
            try:
                if length > 100:
                    new_comment = ' '.join(temp[:100])
                    sent = sentiment_pipeline(new_comment)
                    col_list.append(sent[0])

                else:
                    sent = sentiment_pipeline(comment)
                    col_list.append(sent[0])
            except RuntimeError:
                continue
        
        #
        moodDF.loc[:, col] = pd.Series(col_list)
        
        
    #cleaning moodDF, getting games out if there arent many comments
    new_moodDF = moodDF.copy()
    for column in new_moodDF.columns: 
        if new_moodDF[column].count() <= 7: 
            new_moodDF=new_moodDF.drop(column, axis=1) 
    
    #now making percentage
    percentage = pd.DataFrame()

    for col in new_moodDF.columns:
        num = 0
        num_negative = 0
        for item in new_moodDF[col].dropna():
            if item['label'] == 'NEGATIVE':
                num_negative += 1
            num += 1
        percentage[col] = [(num_negative/num)]
    
    return percentage


In [8]:
# liverpool_data2223 = get_comment_data(end_of_season_2223, beginning_of_season2223, submission_list_liverpool)
# liverpool_data2223.to_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\liverpool.csv')
#Above was done earlier and let run, but now I am accessing through the download
liverpool_data2223 = pd.read_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\liverpool.csv')
liverpool_data2223 = liverpool_data2223.drop('Unnamed: 0', axis=1)
liverpool_mood = get_mood_ratio(liverpool_data2223)
liverpool_mood


Token indices sequence length is longer than the specified maximum sequence length for this model (893 > 512). Running this sequence through the model will result in indexing errors


,FT Thread: Liverpool 7-0 Man United,FT Thread: Liverpool 2-5 Real Madrid,FT Thread: Palace 0-0 Liverpool,FT Thread - Liverpool 2-0 Wolves,FT Thread: Newcastle 0-2 Liverpool,FT Thread: Liverpool 2-0 Everton,FT Thread: Liverpool 0-0 Chelsea,FT Thread: Brighton 3-0 Liverpool,FT Thread: Brentford 3-1 Liverpool FC,FT Thread: Liverpool FC 2-2 Wolves,...,FT Thread: AFC Ajax 0-3 Liverpool FC,FT Thread: Rangers FC 1-7 Liverpool FC,FT Thread - Liverpool 2-1 Ajax,FT Thread: Everton FC 0-0 Liverpool FC,FT Thread: United 2-1 Liverpool,FT Thread: Liverpool FC 2-1 Newcastle United FC,FT Thread: Liverpool FC 2-0 Rangers FC,FT Thread: Liverpool FC 1-1 Crystal Palace,FT Thread: Liverpool FC 9-0 AFC Bournemouth,FT Thread: Fulham FC 2-2 Liverpool FC
0,0.517186,0.723953,0.730477,0.541772,0.521893,0.443812,0.734157,0.76425,0.750269,0.729193,...,0.519924,0.510526,0.537429,0.719119,0.802363,0.604181,0.512461,0.711063,0.548578,0.675081


In [14]:
liverpool_mood.to_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\liverpoolmood.csv')


In [9]:
# arsenal_data2223 = get_comment_data(end_of_season_2223, beginning_of_season2223, submission_list_arsenal)
# arsenal_data2223.to_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\arsenal.csv')
#Above was done earlier and let run, but now I am accessing through the download
arsenal_data2223 = pd.read_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\arsenal.csv')
arsenal_data2223 = arsenal_data2223.drop('Unnamed: 0', axis=1)
arsenal_mood = get_mood_ratio(arsenal_data2223)
arsenal_mood

,Post Match Thread: Arsenal 3 - 2 AFC Bournemouth [English Premier League],Post Match Thread: Leicester City 0 - 1 Arsenal [English Premier League],Post Match Thread: Arsenal 4 - 0 Everton [English Premier League],Post Match Thread: Aston Villa 2 - 4 Arsenal [English Premier League],Post Match Thread: Arsenal 1 - 3 Manchester City [English Premier League],Post Match Thread: Sporting CP 2 - 2 Arsenal [UEFA Europa League],Post Match Thread: Arsenal 3 - 2 Manchester United [English Premier League],Post Match Thread: Everton 1 - 0 Arsenal [English Premier League],Post Match Thread: Arsenal 1 - 1 Brentford [English Premier League],Post Match Thread: Tottenham Hotspur 0 - 2 Arsenal [English Premier League],...,Post Match Thread: Arsenal 5 - 0 Nottingham Forest [English Premier League],Post-Match Thread: Leeds United 0-1 Arsenal,Post Match Thread: Arsenal 2-1 AC Milan (ARS 4-3 penalties),Post Match Thread: Southampton 1 - 1 Arsenal [English Premier League],Post Match Thread: Arsenal 3 - 1 Tottenham Hotspur [English Premier League],Post Match Thread: PSV Eindhoven 2 - 0 Arsenal [UEFA Europa League],Post Match Thread: Arsenal 1 - 0 FC Zürich [UEFA Europa League],Post Match Thread: Arsenal 1 - 0 PSV Eindhoven [UEFA Europa League],Post Match Thread: Arsenal 1 - 3 Brighton & Hove Albion [English Carabao Cup],Post Match Thread: Arsenal 2 - 1 Fulham [English Premier League]
0,0.514113,0.532841,0.439067,0.513491,0.665726,0.63012,0.409677,0.702419,0.703292,0.569546,...,0.461879,0.620166,0.575163,0.774387,0.540636,0.761266,0.631509,0.544209,0.647295,0.4625


In [15]:
arsenal_mood.to_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\arsenalmood.csv')


In [10]:
# manunited_data2223 = get_comment_data(end_of_season_2223, beginning_of_season2223, submission_list_manunited)
# manunited_data2223.to_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\manunited.csv')
#Above was done earlier and let run, but now I am accessing through the download
manunited_data2223 = pd.read_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\manunited.csv')
manunited_data2223 = manunited_data2223.drop('Unnamed: 0', axis=1)
manunited_mood = get_mood_ratio(manunited_data2223)
manunited_mood

,[Post Match Thread] Liverpool 7 - 0 Manchester United,[Post Match Thread] Manchester United 2 - 1 Barcelona [United win 4 - 3 on Aggregate],[Post Match Thread] Manchester United 3 - 1 West Ham United,"[Post Match Interview] Ten Hag: ""You can lose a game but not in this way"" ""The second half is unprofessional, it can't happen. We have to talk about it.""","[Post-Match Quotes] Roy Keane: ""If you lose 7-0 you’d go into hiding, never mind for a few days. I think I’d go missing for a few months it really is that embarrassing for the players."" “Thank god I've never been part of a team that's been beaten by that much playing for Man United.”",[Post Match Thread] Barcelona 2 - 2 Manchester United,[Post Match Thread] Manchester United 3 - 0 Leicester City,[Post-Match Thread] Leeds United 0 - 2 Manchester United,[Post Match Image/Meme/Appreciation Thread] Manchester United 2 - 1 Barcelona,[Post Match Image/Meme/Appreciation Thread] Manchester United 2-0 Newcastle United,...,Manchester United vs Everton - Fa Cup - 6th January 2023 - Post Match Stats [Via Footystats.org],Erik ten Hag FULL post-match press conference | Wolverhampton Wanderers 0-1 Manchester United,[Post Match Thread] Manchester United 1 - 2 Brighton & Hove Albion,"[Post Match Quotes] Ten Hag: ""The result is clear, it never lies. In football you always get your own earnings. Today we were not good enough."" #MUFC",[Post Match Thread] Brentford 4 – 0 Manchester United,Erik ten Hag post-match interview with NBC Sport,[Post Match Thread] Southampton 0 - 1 Manchester United,Luke Shaw post match interview with BT sport,Erik ten Hag EMBARGOED post-match press conference Manchester United 3-1 Everton,Erik ten Hag FULL post-match press conference Manchester United 3-0 Bournemouth
0,0.747026,0.404997,0.497272,0.716049,0.686567,0.514247,0.39267,0.507474,0.527383,0.507732,...,0.5625,0.689655,0.746746,0.707792,0.749587,0.5,0.583854,0.352941,0.928571,0.285714


In [16]:
manunited_mood.to_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\manunitedmood.csv')


In [11]:
# mancity_data2223 = get_comment_data(end_of_season_2223, beginning_of_season2223, submission_list_mancity)
# mancity_data2223.to_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\mancity.csv')
#Above was done earlier and let run, but now I am accessing through the download
mancity_data2223 = pd.read_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\mancity.csv')
mancity_data2223 = mancity_data2223.drop('Unnamed: 0', axis=1)
mancity_mood = get_mood_ratio(mancity_data2223)
mancity_mood

,[Post Match Thread] RB Leipzig 1 - 1 Manchester City,[Post Match Thread] Manchester City 2 - 0 Newcastle,[Post Match Thread] Arsenal 1 - 3 Manchester City,[Post Match Thread] Bournemouth 1 - 4 Manchester City,[Post Match Thread] Bristol City 0 - 3 Manchester City,[Post Match Thread] Tottenham 1 - 0 Manchester City,[Post Match Thread] Manchester United 2 - 1 Manchester City,[Post Match Thread] Manchester City 1 - 0 Arsenal,[Post Match Thread] Southampton 2 - 0 Manchester City,[Post Match Thread] Manchester City 3 - 0 Wolves,...,[Post Match Thread] Manchester City 3 - 1 Brighton,[Post Match Thread] Manchester City 2 - 1 Borussia Dortmund,[Post Match Thread] Manchester City 4 - 0 Southampton,[Post Match Thread] Manchester City 5 - 0 FC Copenhagen,[Post Match Thread] Manchester City 4 - 2 Crystal Palace,[Post Match Thread] Aston Villa 1 - 1 Manchester City,[Post Match Thread] Wolves 0 - 3 Manchester City,[Post Match Thread] Manchester City 6 - 0 Nottingham Forest,[Post Match Thread] Sevilla 0 - 4 Manchester City,[Post Match Thread] West Ham 0 - 2 Manchester City
0,0.746812,0.46875,0.487472,0.57085,0.557143,0.761384,0.819672,0.565359,0.751208,0.616438,...,0.71875,0.619938,0.507937,0.410256,0.501475,0.690955,0.464968,0.405405,0.5,0.453416


In [17]:
mancity_mood.to_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\mancitymood.csv')


In [12]:
# spurs_data2223 = get_comment_data(end_of_season_2223, beginning_of_season2223, submission_list_spurs)
# spurs_data2223.to_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\spurs.csv')
#Above was done earlier and let run, but now I am accessing through the download
spurs_data2223 = pd.read_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\spurs.csv')
spurs_data2223 = spurs_data2223.drop('Unnamed: 0', axis=1)
spurs_mood = get_mood_ratio(spurs_data2223)
spurs_mood

,Post-Match Thread: Wolves vs. Spurs (4 Mar 2023),Post-Match Thread: Sheffield United vs. Spurs (1 Mar 2023),Post-Match Thread: Spurs vs. Chelsea (26 Feb 2023),Post-Match Thread: Spurs vs. West Ham (19 Feb 2023),Post-Match Thread: Leicester City vs. Spurs (11 Feb 2023),Post-Match Thread: Man City vs. Spurs (19 Jan 2023),Post-Match Thread: Spurs vs. Arsenal (15 Jan 2023),Post-Match Thread: Spurs vs. Aston Villa,Post-Match Thread: Fulham vs. Spurs (23 Jan 2023),Post-Match Thread: Preston North End vs. Spurs (28 Jan 2023),...,[Post Match Thread] Brighton Vs. Tottenham Hotspur (PL 08/10/22),[Post-Match Thread] Spurs [6] - 2 Leicester,[Post Match] Frankfurt Vs. Tottenham Hotspur (CL 04/10/22),Post-Match Thread: Chelsea vs. Spurs (14 Aug 2022),Post-Match Thread: Sporting Lisbon vs. Spurs (13 Sep 2022),Post-Match Thread: West Ham vs. Spurs (31 Aug 2022),Post-Match Thread: Spurs vs. Olympique de Marseille (7 Sep 2022),Post-Match Thread: Nottingham Forest vs. Spurs (28 Aug 2022),Post-Match Thread: Spurs vs. Wolves (20 Aug 2022),Post-Match Thread: Spurs vs. Fulham (3 Sep 2022)
0,0.767991,0.783029,0.431579,0.508443,0.779817,0.734358,0.716129,0.758734,0.460843,0.485714,...,0.457249,0.483444,0.725895,0.574651,0.758324,0.745435,0.607407,0.59312,0.471536,0.42053


In [18]:
spurs_mood.to_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\spursmood.csv')


In [13]:
# chelsea_data2223 = get_comment_data(end_of_season_2223, beginning_of_season2223, submission_list_chelsea)
# chelsea_data2223.to_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\chelsea.csv')
#Above was done earlier and let run, but now I am accessing through the download
chelsea_data2223 = pd.read_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\chelsea.csv')
chelsea_data2223 = chelsea_data2223.drop('Unnamed: 0', axis=1)
chelsea_mood = get_mood_ratio(chelsea_data2223)
chelsea_mood

,[Post-Match thread] Tottenham 2-0 Chelsea - Premier League (Matchday 25),Post Match: Chelsea 2 : 0 Borussia Dortmund [Champions League],[Post-Match thread] Chelsea 1-0 Leeds - Premier League (Matchday 26),[Post-Match thread] Chelsea 0-1 Southampton - Premier League (Matchday 24),[Post-Match thread] Borussia Dortmund 1-0 Chelsea - UEFA Champions League (Round of 16),[Post-Match thread] Chelsea 0-0 Fulham - Premier League (Matchday 22),[Post Match Thread] West Ham 1-1 Chelsea | EPL,[Post-Match thread] Liverpool 0-0 Chelsea - Premier League (Matchday 21),[Post Match Thread] Fulham vs Chelsea - EPL,[Post-Match thread] Manchester City 4-0 Chelsea - FA Cup (3rd Round),...,[Post-Match thread] Chelsea 1-1 Red Bull Salzburg - UEFA Champions League (Group E - 2),[Post-Match thread] Chelsea 3-0 AC Milan - UEFA Champions League (Group E - 3),[Post-Match thread] AC Milan 0-2 Chelsea - UEFA Champions League (Group E - 4),Post-Match Thread: West Ham 1 - 1 Chelsea [Premier League],[Post-Match thread] Southampton 2-1 Chelsea - Premier League (Matchday 5),[Post-Match thread] Chelsea 2-1 West Ham - Premier League (Matchday 6),[Post-Match thread] Aston Villa 0-2 Chelsea - Premier League (Matchday 11),[Post-Match thread] Chelsea 2-1 Leicester - Premier League-Matchday 4,[Post-Match thread] Chelsea vs Tottenham ( Matchday 2 ),POST MATCH THREAD : LEEDS VS CHELSEA
0,0.743066,0.43838,0.571264,0.734136,0.626443,0.683942,0.703057,0.579114,0.740032,0.708599,...,0.692884,0.493404,0.537671,0.666667,0.773227,0.648822,0.55,0.523057,0.727165,0.779359


In [19]:
chelsea_mood.to_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\chelseamood.csv')
